In [1]:
from tensorflow.keras.layers import Input, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os
import time
import matplotlib.pyplot as plt
import pickle
import gc
from concurrent.futures import ThreadPoolExecutor
import shutil
from tensorflow.keras import backend as K
import tensorflow as tf
import numpy as np
from PIL import Image
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, concatenate, Input, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose, Activation, Add
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import img_to_array, load_img

KeyboardInterrupt: 

# Préparation Global

In [ ]:
# Configuration
train_limit = 6
val_limit = 2
test_limit = 1
max_visualizations = 5  # Limite d'exemples à afficher

input_dir_rgb = 'P8_Cityscapes_leftImg8bit_trainvaltest'
input_dir_masks = 'P8_Cityscapes_gtFine_trainvaltest'
output_dir = 'processed_data'

# Créer les dossiers de sortie
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, 'train', 'images'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'train', 'masks'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'val', 'images'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'val', 'masks'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'test', 'images'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'test', 'masks'), exist_ok=True)

# Mapping des classes `labelIds` vers les 8 classes cibles
label_mapping = {
    # Flat
    7: 0,  # road
    8: 0,  # sidewalk
    
    # Human
    24: 1,  # person
    25: 1,  # rider
    
    # Vehicle
    26: 2,  # car
    27: 2,  # truck
    28: 2,  # bus
    31: 2,  # train
    32: 2,  # motorcycle
    33: 2,  # bicycle
    
    # Construction
    11: 3,  # building
    12: 3,  # wall
    13: 3,  # fence
    
    # Object
    17: 4,  # pole
    18: 4,  # traffic light
    19: 4,  # traffic sign
    
    # Nature
    21: 5,  # vegetation
    22: 5,  # terrain
    
    # Sky
    23: 6,  # sky
    
    # Void
    0: 7,  # void
    1: 7,  # ego vehicle
    2: 7,  # rectification border
    3: 7,  # out of roi
    4: 7,  # static
    5: 7,  # dynamic
    6: 7,  # ground
    9: 7,  # parking
    10: 7,  # rail track
    14: 7,  # guard rail
    15: 7,  # bridge
    16: 7,  # tunnel
    20: 7,  # polegroup
    29: 7,  # caravan
    30: 7,  # trailer
    -1: 7   # Ignore
}

def remap_labels(mask, mapping):
    remapped_mask = np.copy(mask)
    for original_value, new_value in mapping.items():
        remapped_mask[mask == original_value] = new_value
    remapped_mask[~np.isin(mask, list(mapping.keys()))] = 7  # Assurez-vous que les autres valeurs soient Void
    return remapped_mask

def process_city(city_dir, mask_dir, output_subdir, visualize=False):
    city_name = os.path.basename(city_dir)
    rgb_images = sorted(os.listdir(city_dir))
    
    for img_file in rgb_images:
        img_id = img_file.replace('_leftImg8bit.png', '')
        
        # Charger l'image RGB
        rgb_path = os.path.join(city_dir, img_file)
        rgb_image = Image.open(rgb_path)
        
        # Charger le masque `labelIds`
        mask_label_path = os.path.join(mask_dir, f"{img_id}_gtFine_labelIds.png")
        mask_label = np.array(Image.open(mask_label_path))
        
        # Remapper les labels du masque
        mask_label = remap_labels(mask_label, label_mapping)

        # Sauvegarder les images et les masques
        rgb_image.save(os.path.join(output_dir, output_subdir, 'images', f"{img_id}_image.png"))
        Image.fromarray(mask_label).save(os.path.join(output_dir, output_subdir, 'masks', f"{img_id}_mask.png"))

# Fonction pour traiter un ensemble d'images
def process_dataset(subset_name, limit, visualize=False):
    subset_rgb_dir = os.path.join(input_dir_rgb, subset_name)
    subset_mask_dir = os.path.join(input_dir_masks, subset_name)
    cities = sorted(os.listdir(subset_rgb_dir))[:limit]
    
    with ThreadPoolExecutor(max_workers=4) as executor:
        for city in cities:
            city_rgb_dir = os.path.join(subset_rgb_dir, city)
            city_mask_dir = os.path.join(subset_mask_dir, city)
            executor.submit(process_city, city_rgb_dir, city_mask_dir, subset_name, visualize)

# Traiter les ensembles train, val, et test avec visualisation pour validation
process_dataset('train', train_limit, visualize=True)
process_dataset('val', val_limit, visualize=True)
process_dataset('test', test_limit, visualize=True)

# Préparation des données Sans Data Augmentation

In [2]:
# Class DataGenerator
class DataGenerator:
    def __init__(self, image_files, label_files, batch_size=4, image_size=(256, 256), n_classes=8, augment=False, limit=None):
        if limit:
            self.image_files = image_files[:limit]
            self.label_files = label_files[:limit]
        else:
            self.image_files = image_files
            self.label_files = label_files
        self.batch_size = batch_size
        self.image_size = image_size
        self.n_classes = n_classes
        self.augment = augment

    def load_and_preprocess_image(self, image_path, label_path):
        try:
            image = img_to_array(load_img(image_path, target_size=self.image_size)) / 255.0
            mask = np.array(load_img(label_path, target_size=self.image_size, color_mode="grayscale"))
            mask = np.eye(self.n_classes)[mask]
            return image.astype('float32'), mask.astype('float32')
        except Exception as e:
            print(f"Error loading {image_path} or {label_path}: {e}")
            return None, None

    def create_dataset(self):
        dataset = tf.data.Dataset.from_tensor_slices((self.image_files, self.label_files))
        dataset = dataset.map(lambda img, lbl: tf.numpy_function(self.load_and_preprocess_image, [img, lbl], [tf.float32, tf.float32]), num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.map(lambda img, mask: (tf.ensure_shape(img, [self.image_size[0], self.image_size[1], 3]), 
                                                 tf.ensure_shape(mask, [self.image_size[0], self.image_size[1], self.n_classes])), 
                              num_parallel_calls=tf.data.AUTOTUNE)
        if self.augment:
            dataset = dataset.map(lambda img, mask: self.augment_image(img, mask), num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.batch(self.batch_size)
        dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
        return dataset


In [4]:
# Définir les chemins vers les dossiers de données prétraitées
data_dir = 'C:/Users/DELL/Desktop/OpenClass/Formation/Projet_008/processed_data'

# Charger les chemins des images et des masques pour l'entraînement et la validation
train_image_files = [os.path.join(data_dir, 'train', 'images', f) for f in sorted(os.listdir(os.path.join(data_dir, 'train', 'images'))) if f.endswith('.png')]
train_label_files = [os.path.join(data_dir, 'train', 'masks', f) for f in sorted(os.listdir(os.path.join(data_dir, 'train', 'masks'))) if f.endswith('.png')]

val_image_files = [os.path.join(data_dir, 'val', 'images', f) for f in sorted(os.listdir(os.path.join(data_dir, 'val', 'images'))) if f.endswith('.png')]
val_label_files = [os.path.join(data_dir, 'val', 'masks', f) for f in sorted(os.listdir(os.path.join(data_dir, 'val', 'masks'))) if f.endswith('.png')]

# Définir la taille de l'ensemble de données pour les tests
data_limit = 20 

# Créer les datasets avec DataGenerator
train_generator = DataGenerator(train_image_files, train_label_files, batch_size=2, image_size=(256, 256), n_classes=8, augment=False, limit=100).create_dataset()
train_generator = train_generator.take(20)
val_generator = DataGenerator(val_image_files, val_label_files, batch_size=2, image_size=(256, 256), n_classes=8, augment=False, limit=data_limit).create_dataset()

In [ ]:
# Fonction pour calculer le Dice Loss
def dice_loss(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)

# Fonction pour calculer l'IoU corrigé
def iou(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1, 2, 3])
    union = K.sum(y_true, axis=[1, 2, 3]) + \
        K.sum(y_pred, axis=[1, 2, 3]) - intersection
    return K.mean((intersection + 1e-6) / (union + 1e-6), axis=0)

## Model Unet_mini

In [ ]:
def unet_mini(input_size=(256, 256, 3)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(0.5)(pool1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(0.5)(pool2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = Dropout(0.5)(conv3)

    up4 = UpSampling2D(size=(2, 2))(pool3)
    merge4 = concatenate([conv2, up4], axis=3)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(merge4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)

    up5 = UpSampling2D(size=(2, 2))(conv4)
    merge5 = concatenate([conv1, up5], axis=3)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(merge5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(8, 1, activation='softmax')(conv5)  # 8 classes

    model = Model(inputs, conv5)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss=dice_loss, metrics=[iou])

    return model

def train_mini(learning_rate, image_files_train, label_files_train, image_files_val, label_files_val, image_files_test, label_files_test):
    batch_size = 4
    print(f"Training with learning rate: {learning_rate}")
    
    train_gen = DataGenerator(image_files_train, label_files_train, batch_size=batch_size).create_dataset()
    val_gen = DataGenerator(image_files_val, label_files_val, batch_size=batch_size).create_dataset()
    test_gen = DataGenerator(image_files_test, label_files_test, batch_size=batch_size).create_dataset()

    model = unet_mini()
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss=dice_loss, metrics=[iou])

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
    
    checkpoint_dir = f"lr_{learning_rate}"
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    checkpoint_path = os.path.join(checkpoint_dir, "checkpoint.keras")
    model_checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    
    start_time = time.time()
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=35,
        verbose=1,
        callbacks=[model_checkpoint, reduce_lr, early_stopping]  # Ajout d'EarlyStopping ici
    )
    end_time = time.time()

    with open(f'history_lr_{learning_rate}.pkl', 'wb') as f:
        pickle.dump(history.history, f)
        
    train_iou = history.history['iou'][-1]
    val_iou = history.history['val_iou'][-1]
    
    model.load_weights(checkpoint_path)
    
    loss, test_iou = model.evaluate(test_gen, verbose=1)
    
    print(f"Training Time for learning rate {learning_rate}: {end_time - start_time:.2f} seconds")
    print(f"Training IoU for learning rate {learning_rate}: {train_iou:.2f}")
    print(f"Val IoU for learning rate {learning_rate}: {val_iou:.2f}")
    print(f"Test IoU for learning rate {learning_rate}: {test_iou:.2f}")
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.save(f"mini_unet_LR_{learning_rate}.keras")
    shutil.rmtree(checkpoint_dir)
    
    # clear memory
    K.clear_session()
    gc.collect()

learning_rates = [1e-4]

for lr in learning_rates:
    train_mini(
        learning_rate=lr,
        image_files_train=train_image_files,
        label_files_train=train_label_files,
        image_files_val=val_image_files,
        label_files_val=val_label_files,
        image_files_test=val_image_files,  # Utiliser les fichiers de validation comme test
        label_files_test=val_label_files
    )


In [ ]:
# Charger le modèle sauvegardé
model_path = 'mini_unet2_LR_0.0001.keras'  # Remplacez par le chemin de votre modèle
model = tf.keras.models.load_model(model_path, custom_objects={'dice_loss': dice_loss, 'iou': iou})

def create_jet_palette(num_classes=8):
    cmap = plt.get_cmap('jet', num_classes)
    palette = {i: np.array(cmap(i)[:3]) * 255 for i in range(num_classes)}
    return palette

def color_mask(mask, palette):
    color_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    for label, color in palette.items():
        color_mask[mask == label] = color
    return color_mask

# Créer la palette jet pour 8 classes
palette = create_jet_palette(num_classes=8)

# Choisir une image de validation spécifique
image_index = 0  # Choisissez l'indice de l'image

# Charger l'image d'entrée et le masque de référence (labelIds.png)
input_image, true_mask = next(iter(val_generator))
input_image = input_image.numpy()[image_index]  # Récupérer l'image en tant que NumPy array
input_image_rescaled = (input_image * 255).astype(np.uint8)  # Rescale les valeurs pour être dans [0, 255]
true_mask = true_mask[image_index]

# Colorer le masque de référence
true_mask_resized = np.argmax(true_mask, axis=-1)
true_mask_color = color_mask(true_mask_resized, palette)

# Effectuer la prédiction
predicted_mask = model.predict(np.expand_dims(input_image, axis=0))[0]
predicted_mask_resized = np.argmax(predicted_mask, axis=-1)

# Colorer la prédiction pour la comparer
colored_predicted_mask = color_mask(predicted_mask_resized, palette)

# Afficher les résultats
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.imshow(input_image_rescaled)
plt.title('Input Image')

plt.subplot(1, 3, 2)
plt.imshow(true_mask_color)
plt.title('True Mask (labelIds converted to color)')

plt.subplot(1, 3, 3)
plt.imshow(colored_predicted_mask)
plt.title('Predicted Mask')

plt.show()


## Model DeepLabV3

In [ ]:
# Fonction pour charger DeepLabV3 avec un backbone ResNet50
def deeplabv3_model(input_size=(256, 256, 3), n_classes=8):
    base_model = ResNet50(
        weights='imagenet', 
        include_top=False, 
        input_tensor=Input(input_size)
    )
    
    # Sortie du backbone
    x = base_model.output
    
    # Remise à l'échelle de la sortie pour correspondre à la taille de l'entrée
    x = UpSampling2D(size=(32, 32))(x) 
    
    # Ajouter un segment de sortie personnalisé
    x = tf.keras.layers.Conv2D(n_classes, (1, 1), activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss=dice_loss, metrics=[iou])
    
    return model

# Modifiez votre fonction de formation pour utiliser le modèle DeepLabV3
def train_mini(learning_rate, image_files_train, label_files_train, image_files_val, label_files_val, image_files_test, label_files_test):
    batch_size = 4
    print(f"Training with learning rate: {learning_rate}")
    
    train_gen = DataGenerator(image_files_train, label_files_train, batch_size=batch_size).create_dataset()
    val_gen = DataGenerator(image_files_val, label_files_val, batch_size=batch_size).create_dataset()
    test_gen = DataGenerator(image_files_test, label_files_test, batch_size=batch_size).create_dataset()

    model = deeplabv3_model()  # Utilisation de DeepLabV3
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss=dice_loss, metrics=[iou])

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

    checkpoint_dir = f"lr_{learning_rate}"
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    checkpoint_path = os.path.join(checkpoint_dir, "checkpoint.keras")
    model_checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    
    start_time = time.time()
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=35,
        verbose=1,
        callbacks=[model_checkpoint, reduce_lr, early_stopping]  # Ajout de EarlyStopping
    )
    end_time = time.time()

    # Sauvegarde du fichier history sans "DataAug"
    with open(f'history_lr_{learning_rate}.pkl', 'wb') as f:
        pickle.dump(history.history, f)
        
    train_iou = history.history['iou'][-1]
    val_iou = history.history['val_iou'][-1]
    
    model.load_weights(checkpoint_path)
    
    loss, test_iou = model.evaluate(test_gen, verbose=1)
    
    print(f"Training Time for learning rate {learning_rate}: {end_time - start_time:.2f} seconds")
    print(f"Training IoU for learning rate {learning_rate}: {train_iou:.2f}")
    print(f"Val IoU for learning rate {learning_rate}: {val_iou:.2f}")
    print(f"Test IoU for learning rate {learning_rate}: {test_iou:.2f}")
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.save(f"deeplabv3_LR_{learning_rate}.keras")
    shutil.rmtree(checkpoint_dir)
    
    # clear memory
    K.clear_session()
    gc.collect()

learning_rates = [1e-4]

for lr in learning_rates:
    train_mini(
        learning_rate=lr,
        image_files_train=train_image_files,
        label_files_train=train_label_files,
        image_files_val=val_image_files,
        label_files_val=val_label_files,
        image_files_test=val_image_files,
        label_files_test=val_label_files
    )

In [ ]:
# Charger l'historique de l'entraînement à partir du fichier pickle sauvegardé
with open(f'history_lr_0.0001.pkl', 'rb') as f:
    history = pickle.load(f)

# Récupérer les données d'IoU et de Dice (Dice est calculé à partir de l'IoU)
epochs = range(1, len(history['iou']) + 1)
iou_train = history['iou']
iou_val = history['val_iou']

# Calculer le coefficient Dice à partir de l'IoU
dice_train = [2 * iou / (1 + iou) for iou in iou_train]
dice_val = [2 * iou / (1 + iou) for iou in iou_val]

# Graphique d'évolution de l'IoU
plt.figure(figsize=(10, 5))
plt.plot(epochs, iou_train, label='IoU Entraînement', marker='o')
plt.plot(epochs, iou_val, label='IoU Validation', marker='o')
plt.title('Évolution de l\'IoU au fil des epochs')
plt.xlabel('Epochs')
plt.ylabel('IoU')
plt.legend()
plt.grid(True)
plt.show()

# Graphique d'évolution du Dice coefficient
plt.figure(figsize=(10, 5))
plt.plot(epochs, dice_train, label='Dice Entraînement', marker='o')
plt.plot(epochs, dice_val, label='Dice Validation', marker='o')
plt.title('Évolution du Dice coefficient au fil des epochs')
plt.xlabel('Epochs')
plt.ylabel('Dice Coefficient')
plt.legend()
plt.grid(True)
plt.show()

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Charger le modèle sauvegardé
model_path = 'deeplabv3_LR_0.0001.keras'  # Remplacez par le chemin de votre modèle
model = tf.keras.models.load_model(model_path, custom_objects={'dice_loss': dice_loss, 'iou': iou})

def create_jet_palette(num_classes=8):
    cmap = plt.get_cmap('jet', num_classes)
    palette = {i: np.array(cmap(i)[:3]) * 255 for i in range(num_classes)}
    return palette

def color_mask(mask, palette):
    color_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    for label, color in palette.items():
        color_mask[mask == label] = color
    return color_mask

# Créer la palette jet pour 8 classes
palette = create_jet_palette(num_classes=8)

# Choisir une image de validation spécifique
image_index = 0  # Choisissez l'indice de l'image

# Charger l'image d'entrée et le masque de référence (labelIds.png)
input_image, true_mask = next(iter(val_generator))
input_image = input_image.numpy()[image_index]  # Récupérer l'image en tant que NumPy array
input_image_rescaled = (input_image * 255).astype(np.uint8)  # Rescale les valeurs pour être dans [0, 255]
true_mask = true_mask[image_index]

# Colorer le masque de référence
true_mask_resized = np.argmax(true_mask, axis=-1)
true_mask_color = color_mask(true_mask_resized, palette)

# Effectuer la prédiction
predicted_mask = model.predict(np.expand_dims(input_image, axis=0))[0]
predicted_mask_resized = np.argmax(predicted_mask, axis=-1)

# Colorer la prédiction pour la comparer
colored_predicted_mask = color_mask(predicted_mask_resized, palette)

# Afficher les résultats
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.imshow(input_image_rescaled)
plt.title('Input Image')

plt.subplot(1, 3, 2)
plt.imshow(true_mask_color)
plt.title('True Mask (labelIds converted to color)')

plt.subplot(1, 3, 3)
plt.imshow(colored_predicted_mask)
plt.title('Predicted Mask')

plt.show()


# Préparation Avec Data Augmentation

In [ ]:
# Class DataGenerator
class DataGenerator:
    def __init__(self, image_files, label_files, batch_size=4, image_size=(256, 256), n_classes=8, augment=False, limit=None):
        if limit:
            self.image_files = image_files[:limit]
            self.label_files = label_files[:limit]
        else:
            self.image_files = image_files
            self.label_files = label_files
        self.batch_size = batch_size
        self.image_size = image_size
        self.n_classes = n_classes
        self.augment = augment

    def load_and_preprocess_image(self, image_path, label_path):
        try:
            image = img_to_array(load_img(image_path, target_size=self.image_size)) / 255.0
            mask = np.array(load_img(label_path, target_size=self.image_size, color_mode="grayscale"))
            mask = np.eye(self.n_classes)[mask]
            return image.astype('float32'), mask.astype('float32')
        except Exception as e:
            print(f"Error loading {image_path} or {label_path}: {e}")
            return None, None

    def augment_image(self, image, mask):
        # Combine image and mask for consistent transformations
        combined = tf.concat([image, mask], axis=-1)
    
        # Random horizontal flip
        combined = tf.image.random_flip_left_right(combined)
    
        # Random vertical flip
        combined = tf.image.random_flip_up_down(combined)
    
        # Random rotation using TensorFlow's built-in method
        angles = tf.random.uniform([], minval=-0.2, maxval=0.2) 

        # Random zoom (using crop and resize as a replacement for zoom)
        zoom_factor = tf.random.uniform([], minval=0.8, maxval=1.2)
        new_height = tf.cast(self.image_size[0] * zoom_factor, tf.int32)
        new_width = tf.cast(self.image_size[1] * zoom_factor, tf.int32)
        combined = tf.image.resize_with_crop_or_pad(combined, new_height, new_width)
        combined = tf.image.resize(combined, self.image_size)

        # Random brightness adjustment
        image, mask = combined[..., :3], combined[..., 3:]
        image = tf.image.random_brightness(image, max_delta=0.05)
    
        # Random contrast adjustment
        image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    
        # Random saturation and hue adjustment
        image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
        image = tf.image.random_hue(image, max_delta=0.05)
    
        return image, mask

    def create_dataset(self):
        dataset = tf.data.Dataset.from_tensor_slices((self.image_files, self.label_files))
        dataset = dataset.map(lambda img, lbl: tf.numpy_function(self.load_and_preprocess_image, [img, lbl], [tf.float32, tf.float32]), num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.map(lambda img, mask: (tf.ensure_shape(img, [self.image_size[0], self.image_size[1], 3]), 
                                                 tf.ensure_shape(mask, [self.image_size[0], self.image_size[1], self.n_classes])), 
                              num_parallel_calls=tf.data.AUTOTUNE)
        if self.augment:
            dataset = dataset.map(lambda img, mask: self.augment_image(img, mask), num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.batch(self.batch_size)
        dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
        return dataset


In [ ]:
# Définir les chemins vers les dossiers de données prétraitées
data_dir = 'C:/Users/DELL/Desktop/OpenClass/Formation/Projet_008/processed_data'

# Charger les chemins des images et des masques pour l'entraînement et la validation
train_image_files = [os.path.join(data_dir, 'train', 'images', f) for f in sorted(os.listdir(os.path.join(data_dir, 'train', 'images'))) if f.endswith('.png')]
train_label_files = [os.path.join(data_dir, 'train', 'masks', f) for f in sorted(os.listdir(os.path.join(data_dir, 'train', 'masks'))) if f.endswith('.png')]

val_image_files = [os.path.join(data_dir, 'val', 'images', f) for f in sorted(os.listdir(os.path.join(data_dir, 'val', 'images'))) if f.endswith('.png')]
val_label_files = [os.path.join(data_dir, 'val', 'masks', f) for f in sorted(os.listdir(os.path.join(data_dir, 'val', 'masks'))) if f.endswith('.png')]

# Définir la taille de l'ensemble de données pour les tests
data_limit = 20 

# Créer les datasets avec DataGenerator
train_generator = DataGenerator(train_image_files, train_label_files, batch_size=2, image_size=(256, 256), n_classes=8, augment=True, limit=100).create_dataset()
train_generator = train_generator.take(20)
val_generator = DataGenerator(val_image_files, val_label_files, batch_size=2, image_size=(256, 256), n_classes=8, augment=False, limit=data_limit).create_dataset()

In [ ]:
# Fonction pour calculer le Dice Loss
def dice_loss(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)

# Fonction pour calculer l'IoU corrigé
def iou(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1, 2, 3])
    union = K.sum(y_true, axis=[1, 2, 3]) + \
        K.sum(y_pred, axis=[1, 2, 3]) - intersection
    return K.mean((intersection + 1e-6) / (union + 1e-6), axis=0)

## Modèle U-Net_mini

In [ ]:
def unet_mini(input_size=(256, 256, 3)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(0.5)(pool1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(0.5)(pool2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = Dropout(0.5)(conv3)

    up4 = UpSampling2D(size=(2, 2))(pool3)
    merge4 = concatenate([conv2, up4], axis=3)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(merge4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)

    up5 = UpSampling2D(size=(2, 2))(conv4)
    merge5 = concatenate([conv1, up5], axis=3)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(merge5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(8, 1, activation='softmax')(conv5)  # 8 classes

    model = Model(inputs, conv5)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss=dice_loss, metrics=[iou])

    return model

def train_mini(learning_rate, image_files_train, label_files_train, image_files_val, label_files_val, image_files_test, label_files_test):
    batch_size = 4
    print(f"Training with learning rate: {learning_rate}")
    
    train_gen = DataGenerator(image_files_train, label_files_train, batch_size=batch_size).create_dataset()
    val_gen = DataGenerator(image_files_val, label_files_val, batch_size=batch_size).create_dataset()
    test_gen = DataGenerator(image_files_test, label_files_test, batch_size=batch_size).create_dataset()

    model = unet_mini()
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss=dice_loss, metrics=[iou])

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

    checkpoint_dir = f"lr_{learning_rate}"
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    checkpoint_path = os.path.join(checkpoint_dir, "checkpoint.keras")
    model_checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    
    start_time = time.time()
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=35,
        verbose=1,
        callbacks=[model_checkpoint, reduce_lr, early_stopping]  # Ajout d'EarlyStopping
    )
    end_time = time.time()

    # Sauvegarde du fichier history avec le nom modifié
    with open(f'history_lr_{learning_rate}_DataAug.pkl', 'wb') as f:
        pickle.dump(history.history, f)
        
    train_iou = history.history['iou'][-1]
    val_iou = history.history['val_iou'][-1]
    
    model.load_weights(checkpoint_path)
    
    loss, test_iou = model.evaluate(test_gen, verbose=1)
    
    print(f"Training Time for learning rate {learning_rate}: {end_time - start_time:.2f} seconds")
    print(f"Training IoU for learning rate {learning_rate}: {train_iou:.2f}")
    print(f"Val IoU for learning rate {learning_rate}: {val_iou:.2f}")
    print(f"Test IoU for learning rate {learning_rate}: {test_iou:.2f}")
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.save(f"mini_unet2_LR_{learning_rate}_DataAug.keras")
    shutil.rmtree(checkpoint_dir)
    
    # clear memory
    K.clear_session()
    gc.collect()

learning_rates = [1e-4]

for lr in learning_rates:
    train_mini(
        learning_rate=lr,
        image_files_train=train_image_files,
        label_files_train=train_label_files,
        image_files_val=val_image_files,
        label_files_val=val_label_files,
        image_files_test=val_image_files,
        label_files_test=val_label_files
    )

In [ ]:
# Charger le modèle sauvegardé
model_path = 'mini_unet2_LR_0.0001_DataAug.keras'  # Remplacez par le chemin de votre modèle
model = tf.keras.models.load_model(model_path, custom_objects={'dice_loss': dice_loss, 'iou': iou})

def create_jet_palette(num_classes=8):
    cmap = plt.get_cmap('jet', num_classes)
    palette = {i: np.array(cmap(i)[:3]) * 255 for i in range(num_classes)}
    return palette

def color_mask(mask, palette):
    color_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    for label, color in palette.items():
        color_mask[mask == label] = color
    return color_mask

# Créer la palette jet pour 8 classes
palette = create_jet_palette(num_classes=8)

# Choisir une image de validation spécifique
image_index = 0  # Choisissez l'indice de l'image

# Charger l'image d'entrée et le masque de référence (labelIds.png)
input_image, true_mask = next(iter(val_generator))
input_image = input_image.numpy()[image_index]  # Récupérer l'image en tant que NumPy array
input_image_rescaled = (input_image * 255).astype(np.uint8)  # Rescale les valeurs pour être dans [0, 255]
true_mask = true_mask[image_index]

# Colorer le masque de référence
true_mask_resized = np.argmax(true_mask, axis=-1)
true_mask_color = color_mask(true_mask_resized, palette)

# Effectuer la prédiction
predicted_mask = model.predict(np.expand_dims(input_image, axis=0))[0]
predicted_mask_resized = np.argmax(predicted_mask, axis=-1)

# Colorer la prédiction pour la comparer
colored_predicted_mask = color_mask(predicted_mask_resized, palette)

# Afficher les résultats
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.imshow(input_image_rescaled)
plt.title('Input Image')

plt.subplot(1, 3, 2)
plt.imshow(true_mask_color)
plt.title('True Mask (labelIds converted to color)')

plt.subplot(1, 3, 3)
plt.imshow(colored_predicted_mask)
plt.title('Predicted Mask')

plt.show()


## Modèle DeeplabV3+

In [ ]:
# Fonction pour charger DeepLabV3 avec un backbone ResNet50
def deeplabv3_model(input_size=(256, 256, 3), n_classes=8):
    base_model = ResNet50(
        weights='imagenet', 
        include_top=False, 
        input_tensor=Input(input_size)
    )
    
    # Sortie du backbone
    x = base_model.output
    
    # Remise à l'échelle de la sortie pour correspondre à la taille de l'entrée
    x = UpSampling2D(size=(32, 32))(x) 
    
    # Ajouter un segment de sortie personnalisé
    x = tf.keras.layers.Conv2D(n_classes, (1, 1), activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss=dice_loss, metrics=[iou])
    
    return model

# Modifiez votre fonction de formation pour utiliser le modèle DeepLabV3
def train_mini(learning_rate, image_files_train, label_files_train, image_files_val, label_files_val, image_files_test, label_files_test):
    batch_size = 4
    print(f"Training with learning rate: {learning_rate}")
    
    train_gen = DataGenerator(image_files_train, label_files_train, batch_size=batch_size).create_dataset()
    val_gen = DataGenerator(image_files_val, label_files_val, batch_size=batch_size).create_dataset()
    test_gen = DataGenerator(image_files_test, label_files_test, batch_size=batch_size).create_dataset()

    model = deeplabv3_model()  # Utilisation de DeepLabV3
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss=dice_loss, metrics=[iou])

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

    checkpoint_dir = f"lr_{learning_rate}"
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    checkpoint_path = os.path.join(checkpoint_dir, "checkpoint.keras")
    model_checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    
    start_time = time.time()
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=35,
        verbose=1,
        callbacks=[model_checkpoint, reduce_lr, early_stopping] 
    )
    end_time = time.time()

    # Sauvegarde du fichier history avec "DataAug" ajouté
    with open(f'history_lr_{learning_rate}_DataAug.pkl', 'wb') as f:
        pickle.dump(history.history, f)
        
    train_iou = history.history['iou'][-1]
    val_iou = history.history['val_iou'][-1]
    
    model.load_weights(checkpoint_path)
    
    loss, test_iou = model.evaluate(test_gen, verbose=1)
    
    print(f"Training Time for learning rate {learning_rate}: {end_time - start_time:.2f} seconds")
    print(f"Training IoU for learning rate {learning_rate}: {train_iou:.2f}")
    print(f"Val IoU for learning rate {learning_rate}: {val_iou:.2f}")
    print(f"Test IoU for learning rate {learning_rate}: {test_iou:.2f}")
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.save(f"deeplabv3_LR_{learning_rate}_DataAug.keras")
    shutil.rmtree(checkpoint_dir)
    
    # clear memory
    K.clear_session()
    gc.collect()

learning_rates = [1e-4]

for lr in learning_rates:
    train_mini(
        learning_rate=lr,
        image_files_train=train_image_files,
        label_files_train=train_label_files,
        image_files_val=val_image_files,
        label_files_val=val_label_files,
        image_files_test=val_image_files, 
        label_files_test=val_label_files)

In [ ]:
# Charger l'historique de l'entraînement à partir du fichier pickle sauvegardé
with open(f'history_lr_0.0001_DataAug.pkl', 'rb') as f:
    history = pickle.load(f)

# Récupérer les données d'IoU et de Dice (Dice est calculé à partir de l'IoU)
epochs = range(1, len(history['iou']) + 1)
iou_train = history['iou']
iou_val = history['val_iou']

# Calculer le coefficient Dice à partir de l'IoU
dice_train = [2 * iou / (1 + iou) for iou in iou_train]
dice_val = [2 * iou / (1 + iou) for iou in iou_val]

# Graphique d'évolution de l'IoU
plt.figure(figsize=(10, 5))
plt.plot(epochs, iou_train, label='IoU Entraînement', marker='o')
plt.plot(epochs, iou_val, label='IoU Validation', marker='o')
plt.title('Évolution de l\'IoU au fil des epochs')
plt.xlabel('Epochs')
plt.ylabel('IoU')
plt.legend()
plt.grid(True)
plt.show()

# Graphique d'évolution du Dice coefficient
plt.figure(figsize=(10, 5))
plt.plot(epochs, dice_train, label='Dice Entraînement', marker='o')
plt.plot(epochs, dice_val, label='Dice Validation', marker='o')
plt.title('Évolution du Dice coefficient au fil des epochs')
plt.xlabel('Epochs')
plt.ylabel('Dice Coefficient')
plt.legend()
plt.grid(True)
plt.show()

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Charger le modèle sauvegardé
model_path = 'deeplabv3_LR_0.0001_DataAug.keras'  # Remplacez par le chemin de votre modèle
model = tf.keras.models.load_model(model_path, custom_objects={'dice_loss': dice_loss, 'iou': iou})

def create_jet_palette(num_classes=8):
    cmap = plt.get_cmap('jet', num_classes)
    palette = {i: np.array(cmap(i)[:3]) * 255 for i in range(num_classes)}
    return palette

def color_mask(mask, palette):
    color_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    for label, color in palette.items():
        color_mask[mask == label] = color
    return color_mask

# Créer la palette jet pour 8 classes
palette = create_jet_palette(num_classes=8)

# Choisir une image de validation spécifique
image_index = 0  # Choisissez l'indice de l'image

# Charger l'image d'entrée et le masque de référence (labelIds.png)
input_image, true_mask = next(iter(val_generator))
input_image = input_image.numpy()[image_index]  # Récupérer l'image en tant que NumPy array
input_image_rescaled = (input_image * 255).astype(np.uint8)  # Rescale les valeurs pour être dans [0, 255]
true_mask = true_mask[image_index]

# Colorer le masque de référence
true_mask_resized = np.argmax(true_mask, axis=-1)
true_mask_color = color_mask(true_mask_resized, palette)

# Effectuer la prédiction
predicted_mask = model.predict(np.expand_dims(input_image, axis=0))[0]
predicted_mask_resized = np.argmax(predicted_mask, axis=-1)

# Colorer la prédiction pour la comparer
colored_predicted_mask = color_mask(predicted_mask_resized, palette)

# Afficher les résultats
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.imshow(input_image_rescaled)
plt.title('Input Image')

plt.subplot(1, 3, 2)
plt.imshow(true_mask_color)
plt.title('True Mask (labelIds converted to color)')

plt.subplot(1, 3, 3)
plt.imshow(colored_predicted_mask)
plt.title('Predicted Mask')

plt.show()
